## We solve the MHD toy model with PINN

### The evolution equations are:

$$
\partial_t B^i = \nabla_j (v^j B^i - v^i B^j) \;\;\;\; \nabla_i B^i = 0
$$

The vector field $v^i$ is given. 

And we impose Dirichlet boundary conditions.

The initial data is: 

\begin{align*}
B_1 &= \partial_y \phi \\
B_2 &= -\partial_x \phi
\end{align*}
with (here we use xmin=0, xmax = 1, same for y)

$$
\phi(x,y) = (x*(x-1)*y*(y-1))^2
$$

The velocity field is time-independent and given by:

\begin{align*}
v1(t,x,y) &= \sin(\pi*x)*\cos(\pi*y) \\
v2(t,x,y) &= \cos(\pi*x)*\sin(\pi*y) \\
\end{align*}



In [ ]:
#import Pkg
#Pkg.add(url="https://github.com/psy3nt1st/Optim.jl.git")
using Optim
#using NeuralPDE
#using Optimization
using OptimizationOptimJL
using Roots
using LineSearches
using ModelingToolkit, IntervalSets 
using IntervalSets
using Plots, Printf
using Lux, LuxCUDA, ComponentArrays, Random
using JLD2, LinearAlgebra 
using Revise
using Distributions
using Zygote

const gpud = gpu_device()

includet("../neural_tools.jl")

In [ ]:
#@load "toy_MHD_2D_Man_Ada.jld2" config res st losses

#@load "toy_MHD_2D_Man.jld2" config res st losses

@load "toy_MHD_2D_Man_comp.jld2" config res st losses

NN = create_chain(config)

@show config

config[:method]


In [ ]:
plot(losses, 
    #yscale=:log10, 
    label="loss", title="Loss vs Iterations", xlabel="Iterations", ylabel="Loss")

In [ ]:
minimum(losses)

In [ ]:
@unpack xmin, xmax, ymin, ymax, tmin, tmax = config
xs = collect(xmin:0.005:xmax)
ys = collect(ymin:0.005:ymax);

t = tmax
B1_approx = [NN(vcat(t, x, y), res, st)[1][1] for x in xs, y in ys]
B2_approx = [NN(vcat(t, x, y), res, st)[1][2] for x in xs, y in ys]
p1 = plot(ys, xs, B1_approx, 
        linetype = :contourf, 
        #st = :surface,
        title = "predict B1 at t=$t", aspect_ratio = 1)
p2 = plot(ys, xs, B2_approx, 
        linetype = :contourf, 
        #st = :surface,
        title = "predict B2 at t=$t", aspect_ratio = 1)
ps = [p1, p2]
plot(ps..., layout = (1,2), size = (900,400))


In [ ]:
tmax

In [ ]:
#@load "../../Toy_MHD/toy_MHD_2d_FD.jld2" B B_final dx J Box t_final
@load "../../Toy_MHD/toy_MHD_2d_FD_WBC.jld2" B B_final dx J Box t_final

In [ ]:
B_final_enlarg = zeros(size(B_final)[1], size(B_final)[2]+1, size(B_final)[3]+1)

B_final_enlarg[:,1:end-1,1:end-1] = B_final[:,:,:]
B_final_enlarg[:,1:end-1,end] = B_final[:,:,1]
B_final_enlarg[:,end,1:end-1] = B_final[:,1,:]
B_final_enlarg[:,end,end] = B_final[:,1,1]

B_initial = similar(B_final_enlarg)
B_initial[:,1:end-1,1:end-1] = B[:, :, :]
B_initial[:,1:end-1,end] = B[:,:,1]
B_initial[:,end,1:end-1] = B[:,1,:]
B_initial[:,end,end] = B[:,1,1]


In [ ]:
plot(ys, xs, B_final_enlarg[1,:,:] - 0.0*B1_approx,
        linetype = :contourf, 
        #st = :surface,
        title = "predict B1 at t=$t", aspect_ratio = 1)

In [ ]:
j = J[2]÷2
B1_approx_initial = [NN(vcat(tmin, x, y), res, st)[1][1] for x in xs, y in ys]
#plot(B_final_enlarg[1,j,:], label="FD")
plot(B_final[1,:,j], label="FD t= $t_final")
plot!(B1_approx[:,j], label="PINN")
plot!(B[1,:,j], label="Initial condition")
plot!(B1_approx_initial[:,j], label="PINN Initial condition")
savefig("toy_MHD_bc_out_t4_direct.png")